<a href="https://colab.research.google.com/github/blazingbhavneek/hyperbolic-ml/blob/main/L_Hydra_Cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L-Hydra (Cleaned, with Random Graph)

In [1]:
import numpy as np
import os, math, warnings
import sys, time, pdb, os, pickle
from numpy import *
import networkx as nx
from scipy.linalg import eigh
import sys, time, pdb, os, pickle

##---------- Parameters ----------

In [3]:
nlandmarks = 101
nNodes = 250
curvature = 1
dim = 100
alpha = 1
equi_adj = 0.5

In [21]:
def randX(N,land, d):
  n = N-land
  n_temp = n
  param = 0.4  #distance thresold
  g = nx.generators.random_geometric_graph(n_temp, param, seed = 1)
  W = np.zeros((n_temp, n_temp))
  for (x,y) in g.edges:
    W[x][y] = np.random.uniform(0.1,3)
  W_hat = W + W.T
  L = np.diag(np.matmul(W_hat,np.ones(W_hat.shape[0]))) - W_hat

  np.random.seed(1)
  X_na = np.random.multivariate_normal(np.zeros(n), np.linalg.pinv(L),d+1)

  np.random.seed(5)
  X_a = np.random.normal(0,1,size=(d+1,land))

  X = np.hstack((X_a, X_na))

  for i in range(N):
    X[0,i] = np.sqrt(1+np.linalg.norm(X[1:d+1,i])**2);
  
  return X


def x2hdm(N,land, X,d):
  n = N-land
  G = x2hgram(N,d,X)
  D = _arccosh(G)

  return D 


def _arccosh(G):
  D = np.arccosh(-G)
  return D


def x2hgram(N,d, X):
  n = N-d

  X_ = X
  for n in range(N):
      x = X_[:,n]
      X_[:,n] = projectX(N,d,x)

  G = x2lgram(N,d, X_)

  E1 = G-valid(G) 

  if np.linalg.norm(E1,'fro') > 1e-10:
      print(np.linalg.norm(E1,'fro'))
      print('inaccuracy in  x2hgram - ')

  E2 = G-l_rankPrj(G, N)

  if np.linalg.norm(E2,'fro') > 1e-10:
      print(np.linalg.norm(E2,'fro'))
      print('inaccuracy in  x2hgram -- ')
  return G


def projectX(N,d,x):
  n= N-d
  
  
  H = np.eye(d+1)
  H[0,0] = -1
  I = np.eye(d+1)
  
  x0 = x[0]
  
  eps = 1e-15
  center = 0
  error = abs(h_norm(x,N,d)+1)
  
  A_opt = I
  for i in range(50):
      l = 10**(-i)
      if x0 > 0:
          lambda_min = max(center-l, -1+eps)
          lambda_max = min(center+l, 1-eps)
          number = 50
      else:
          lambda_min = max(center-l*10000, 1+eps)
          lambda_max = center+l*10000
          number = 100
      lambda_list = np.linspace(lambda_min,lambda_max,num=number)
      for lambda_ in lambda_list:
          A = np.linalg.inv(I + lambda_*H)
          x_l = np.matmul(A,x)
          if abs(h_norm(x_l,N,d)+1) < error:
              A_opt = A
              error = abs(h_norm(x_l,N,d)+1)
              center = lambda_
  x_opt = np.matmul(A_opt,x)
  if error > 1e-5:
      print('error: ',error, "centre: ", center)
   
  return x_opt


def x2lgram(N,d,X):
  n= N-d
  
  H = np.eye(d+1)
  H[0,0] = -1
  
  G = np.matmul(np.matmul(X.T,H),X)
  return G


def h_norm(x,N,d):
  n= N-d
  
  H = np.eye(d+1)
  H[0,0] = -1
  
  x_norm = np.matmul(np.matmul(x.T,H),x)
  return x_norm


def valid(G):
  np.fill_diagonal(G, -1)
  G[G >= -1] = -1
  
  return G


def l_rankPrj(G, N):
  X = lgram2x(N,d,G)
  
  return x2lgram(N,d,X)


def lgram2x(N,d,G):
  n= N-d
  
  w, v = np.linalg.eig(G)
  w = w.real
  v = v.real

  lambda_0 = np.amin(w)
  ind_0 = np.argmin(w)
  w = np.delete(w, ind_0)
  ind = np.argsort(-w)
  w = -np.sort(-w)
  ind = ind[:d]
  w = w[:d]
  
  lambda_ = np.concatenate((abs(lambda_0), w), axis=None) 
  lambda_[lambda_ <= 0] = 0
  lambda_ = np.sqrt(lambda_)

  ind_ = np.concatenate((ind_0, ind+1), axis=None) 
  v = v[:, ind_]
  X = np.matmul(np.diag(lambda_),v.T)
  
  if X[0,0] < 0:
      X = -X
  
  return X                    

##---------- Generating Random X and D ----------

In [7]:
#---------- Generating Random X and D ----------
d = dim-1

X = randX(nNodes,nlandmarks, d)
print("X: ",X.shape)

D = x2hdm(nNodes,nlandmarks, X,d)
D = (D+D.T)/2
print("D: ", D.shape)

L2n = D[:nlandmarks,:]
print("L2n: ", L2n.shape)
#---------- ------------------------ ----------

X:  (100, 250)
D:  (250, 250)
L2n:  (101, 250)


In [8]:
np.save('/content/X', X)
np.save('/content/D', D)

### Objective function definitions


In [9]:
# Frobenius Norm
def f(X,Y):
	return .5*linalg.norm(X-Y, ord='fro')**2

# Matrix Difference
def df(X,Y):
	return X-Y

# Hyperbolic Distance
def hype_dist(p1,p2,curv):
	k = curv
	c = 1./sqrt(abs(k))
	x = dot(p1,p2)
	usq1 = dot(p1,p1) + 1.0
	usq2 = dot(p2,p2) + 1.0
	udotu = x - sqrt(usq1)*sqrt(usq2)
	acoshudotu = real(arccosh(-udotu + 0j))
	d = acoshudotu * c
	return d

# Objective func for Xl: Eq 14a
def land_obj(x,**kwargs):
	dim = kwargs['dim']
	D0 = kwargs['D_landmark']
	grad = zeros(len(x))
	points = len(x[:-1])//dim
	k = x[-1]
	c = 1./sqrt(abs(k))
	P = x[:-1].reshape((points,dim))

	X = dot(P,P.T)
	Usq = diag(X) + 1.0
	U = sqrt(Usq)
	UdotU = X - outer(U,U)
	acoshUdotU = real(arccosh(-UdotU + 0j))
	fill_diagonal(acoshUdotU,0.0)

	D = acoshUdotU*c;
	D[isnan(D)] = 0
	dE = df(D,D0)
	err = f(D,D0)/2.0 # since matrix is symmetric

	# compute gradient w.r.t. k
	gradk = - dE * acoshUdotU * .5 * k * abs(k)**-2.5
	grad[-1] = sum(sum(gradk))/2.0 # divide by two since matrix is symmetric

	# compute gradient
	tol = 1e-12
	Atemp = UdotU**2 - 1.0
	Atemp[Atemp<=0] = tol
	A = - 1./sqrt(Atemp) * c
	B = outer(1./U,U)
	C = dE * A
	H1 = C * B
	hsum = sum(H1,1)
	L = len(D0)
	Ci = zeros((len(C),len(C)))
	for i in range(L):
		ui = P[i,:]
		hi = hsum[i]
		fill_diagonal(Ci,C[i])
		g = sum(dot(P.T,Ci),1) - hi*ui
		grad[dim*i:(i+1)*dim] = g

	return err, grad

# Objective func for Xn: Eq 14b
def nonland_obj(x, node, **kwargs):
	'''
	computes nonlandmark error for an entire nonlandmark
	matrix D0
	'''
	# node is index of D_nonland not D_land2nodes
	dim = kwargs['dim']
	D0 = kwargs['D_nonland']
	L = kwargs['Land_points']
	n_land, dim = L.shape
	nl_id = node
	k = kwargs['k']
	c = 1./sqrt(abs(k))

	X = dot(L,x)
	u = sqrt(dot(x,x) + 1.0)
	V = sqrt(sum(L.T**2,0) + 1.0)
	udotV = outer(u,V) - X
	d = real(arccosh(udotV + 0j)) * c
	d0 = D0[:,nl_id]
	err = f(d,d0)
	derr = df(d,d0)

	# compute gradient
	tol = 1e-12
	Atemp = udotV**2 - 1.0
	Atemp[Atemp<=0] = tol
	A = - 1./sqrt(Atemp) * c
	B = outer(1./u,V)
	C = derr * A
	H1 = C * B
	H1.shape = (n_land,1)
	C.shape = (n_land,1)

	ui = x
	Ui = tile(ui,(n_land,1)).T
	Hi = diag(H1[:,0])
	Ci = diag(C[:,0])
	G = dot(L.T,Ci) - dot(Ui,Hi)
	grad = sum(G,1)

	return err, grad

## L_Hydra

In [10]:
land_ids = [i for i in range(nlandmarks)]
non_land_ids = [i for i in range(nlandmarks,nNodes)]
D_land2nodes, D_land, land_ids, D_nonland, nonland_ids = L2n, L2n[:,:nlandmarks], land_ids, L2n[:,nlandmarks:], non_land_ids 

In [11]:
def hydra_landmark_fixed_curvature(curvature, dim, alpha, equi_adj, polar=False, isotropic_adj=True, hydra=False, lorentz=False):

    nodes = nNodes

    # sanitize/check input
    if any(np.diag(D_land) != 0):  # non-zero diagonal elements are set to zero
      np.fill_diagonal(D_land, 0)
      warnings.warn("Diagonal of input matrix D_land has been set to zero")
      
    if dim > len(D_land):
      raise RuntimeError(
        f"Hydra cannot embed {len(D_land)} points in {dim}-dimensions. Limit of {len(D_land)}."
      )
    
    if not np.allclose(D_land, np.transpose(D_land)):
      warnings.warn(
        "Input matrix D_land is not symmetric.\
        Lower triangle part is used."
      )

    if dim > 2:
      # set default values in dimension > 2
      isotropic_adj = False
      if polar:
        warnings.warn("Polar coordinates only valid in dimension two")
        polar = False
      if equi_adj != 0.0:
        warnings.warn("Equiangular adjustment only possible in dimension two.")
      
    # convert distance matrix to 'hyperbolic Gram matrix'
    A_land = np.cosh(np.sqrt(abs(curvature))*D_land)
    A_nonland = np.cosh(np.sqrt(abs(curvature))*D_nonland)

    nlm = len(land_ids)
    nnonlm = nodes - nlm
    
    # check for large/infinite values
    A_max = np.amax(A_land)
    if A_max > 1e8:
      warnings.warn(
        "Gram Matrix contains values > 1e8. Rerun with smaller\
        curvature parameter or rescaled distances."
      )
    if A_max == float("inf"):
      warnings.warn(
        "Gram matrix contains infinite values.\
        Rerun with smaller curvature parameter or rescaled distances."
      )


    # Eigendecomposition of A
    # compute leading Eigenvalue and Eigenvector
    lambda0, x0 = eigh(A_land, subset_by_index=[nlm-1, nlm-1])
    # compute lower tail of spectrum
    w, v = eigh(A_land, subset_by_index=[0,dim-1])
    print("w",len(w))
    print("v", len(v))
    idx = w.argsort()[::-1]
    # print("idx",len(idx))
    spec_tail = w[idx] # Last dim Eigenvalues
    print("")
    X_land_raw = v[:,idx] # Last dim Eigenvectors
    print("shape of  X_land_raw",  X_land_raw.shape) 
    
    x0 = x0 * np.sqrt(lambda0) # scale by Eigenvalue
    if x0[0]<0:
      x0 = -x0 # Flip sign if first element negative

    # no isotropic adjustment: rescale Eigenvectors by Eigenvalues
    if not isotropic_adj:
      if np.array([spec_tail > 0]).any():
        warnings.warn(
          "Spectral Values have been truncated to zero. Try to use\
          lower embedding dimension"
        )
        spec_tail[spec_tail > 0] = 0
      X_land_raw = np.matmul(X_land_raw, np.diag(np.sqrt(np.maximum(-spec_tail,0))))
      
    X_nonland = np.matmul(np.transpose(A_nonland), np.c_[(x0/lambda0), -(X_land_raw/abs(spec_tail))])
    
    X_raw = np.zeros((nodes, dim))

    print(len(land_ids))
    X_raw[land_ids,:] = X_land_raw
    
    X_raw[nonland_ids,:] = X_nonland[:,1:(dim+1)]
    
    x0_full = np.zeros((nodes, 1))
    x0_full[land_ids,0] = x0[:,0]
    x0_full[nonland_ids,0] = X_nonland[:,0]
    x_min = x0_full.min()
    
    
    if hydra:
      # Calculate radial coordinate
      s = np.sqrt(np.sum(X_raw ** 2, axis=1))
      directional = X_raw / s[:, None]  # convert to directional coordinates
      r = np.sqrt((alpha*x0_full - x_min)/(alpha*x0_full + x_min)) ## multiplicative adjustment (scaling)
      X = self.poincare_to_hyper(r=r, directional=directional)
    else:
      X = X_raw
      
    # Calculate polar coordinates if dimension is 2
    if dim == 2:
      # calculate polar angle
      theta = np.arctan2(X[:, 0], -X[:, 1])

      # Equiangular adjustment
      if equi_adj > 0.0:
        angles = [(2 * x / nodes - 1) * math.pi for x in range(0, nodes)]
        theta_equi = np.array(
          [x for _, x in sorted(zip(theta, angles))]
        )  # Equi-spaced angles
        # convex combination of original and equi-spaced angles
        theta = (1 - equi_adj) * theta + equi_adj * theta_equi
        # update directional coordinate
        directional = np.array([np.cos(theta), np.sin(theta)]).transpose()
        
    if lorentz:
      X_lorentz = np.concatenate((x0_full, X), axis=1)

    return X

In [12]:
X_hat = hydra_landmark_fixed_curvature(curvature, dim, alpha, equi_adj)

w 100
v 101

shape of  X_land_raw (101, 100)
101


<ipython-input-11-446413e345e6>:28: UserWarning: Equiangular adjustment only possible in dimension two.
  warnings.warn("Equiangular adjustment only possible in dimension two.")
<ipython-input-11-446413e345e6>:72: UserWarning: Spectral Values have been truncated to zero. Try to use          lower embedding dimension
  warnings.warn(
<ipython-input-11-446413e345e6>:79: RuntimeWarning: invalid value encountered in true_divide
  X_nonland = np.matmul(np.transpose(A_nonland), np.c_[(x0/lambda0), -(X_land_raw/abs(spec_tail))])


In [16]:
X_hat.shape

(250, 100)

#### We need shape of X_hat: [100,250] for x2hdm

In [17]:
X_hat = X_hat.T

In [18]:
X_hat.shape

(100, 250)

In [19]:
np.save('/content/X_hat', X_hat)

In [22]:
D_hat = x2hdm(nNodes,nlandmarks, X_hat,d)

error:  1.0000000000000058 centre:  10838.383838383837
error:  1.0000000000000053 centre:  10838.39393939394
error:  1.0000000000000067 centre:  10858.585858585859
error:  1.000000000000007 centre:  10858.585858585859
error:  1.000000000000007 centre:  10878.787878787878
error:  1.000000000000006 centre:  10757.585858585859
error:  1.000000000000007 centre:  10757.585858585859
error:  1.0000000000000067 centre:  10818.191919191919
error:  1.0000000000000058 centre:  10858.585858585859
error:  1.000000000000007 centre:  10979.797979797979
error:  1.0000000000000089 centre:  10979.797979797979
error:  1.0000000000000056 centre:  10777.787878787878
error:  1.0000000000000064 centre:  10919.191919191919
error:  1.0000000000000075 centre:  11000.0
error:  1.0000000000000067 centre:  10898.9898989899
error:  1.0000000000000087 centre:  10959.59595959596
error:  1.0000000000000062 centre:  10737.383838383837
error:  1.0000000000000078 centre:  10919.191919191919
error:  1.0000000000000082 cen

LinAlgError: ignored

#### Problem with ProjectX, we don't need it anymore, need to redefine x2hdm

In [23]:
def x2hdm_v2(N,land, X,d):
  n = N-land
  G = x2hgram_v2(N,d,X)
  D = _arccosh(G)

  return D 


def x2hgram_v2(N,d, X):
  n = N-d

  G = x2lgram(N,d, X)
  return G

In [24]:
D_hat = x2hdm_v2(nNodes,nlandmarks, X_hat,d)

<ipython-input-21-cd0d0a9876df>:35: RuntimeWarning: invalid value encountered in arccosh
  D = np.arccosh(-G)


In [25]:
D_hat.shape

(250, 250)

In [27]:
np.save('/content/D_hat', D_hat)